# Loguru

## Introducción

Python incluye un práctico módulo de [logging](https://docs.python.org/3/howto/logging.html). Tanto para efectuar una depuración sencilla como para realizar un registro centralizado desde diversos servidores, el módulo logging de Python puede facilitar enormemente el trabajo a los desarrolladores y operadores.

**Logging** proviene del término en inglés “logˮ y, en este contexto, se refiere a un protocolo. Al igual que un libro de registro, contiene todos los registros importantes del historial de eventos. Dependiendo del tipo de seguimiento que queramos hacer, solo se registran ciertas acciones o eventos de un proceso o, por el contrario, se comprueban todas las acciones.

El registro de Python puede contener una gran cantidad de datos, especialmente al desarrollar aplicaciones complejas. Mediante el `logging to file` de Python (es decir, un archivo de registro creado por el módulo logging de Python y en el que un `handler` anota los datos de registro), los desarrolladores recopilan estos datos. Es importante que el archivo de registro funcione de forma asincrónica. De lo contrario, el logging de Python puede bloquear la ejecución del código.

### Análisis de errores

El logging de Python presenta cinco niveles de gravedad distintos, que en inglés reciben el nombre de “levels of severityˮ. Si deseas crear tu propio filtro de registro, obviamente puedes hacerlo, aunque los niveles de gravedad incluidos en el módulo logging de Python, desarrollado por **Vinay Sajip**, nos parecen bastante adecuados:

| Nombre del nivel de registro | Uso                                                                                                                  | Posible salida de mensaje                                                  |
|------------------------------|----------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------|
| Debug                        | Diagnóstico del problema, muy detallado                                                                              | Sangría inesperada en la línea X                                           |
| Info                         | Indica que el sistema funciona correctamente                                                                         | La función 1*1 está ejecutándose.                                          |
| Warning                      | La aplicación funciona correctamente, pero se ha producido una situación inesperada o se predice un problema futuro. | Poco espacio de almacenamiento                                             |
| Error                        | No se pudo realizar una función debido a un problema.                                                                | Ha ocurrido un error y se ha interrumpido la acción.                       |
| Critical                     | Ha ocurrido un problema grave. Es posible que la aplicación deba interrumpirse por completo.                         | Error grave: el programa no puede acceder a este servicio y debe cerrarse. |

* **Debug** es el nivel más bajo, por lo que también genera información de baja prioridad. Esto no significa, sin embargo, que la gravedad de un error sea superior a la de critical.

* Debug incluye todos los demás niveles y, por lo tanto, genera todos los mensajes hasta los de nivel critical.

### Componenetes principales del módulo logging

#### a) Logger

Los logger registran las **acciones durante la ejecución de un programa**. No se pueden usar directamente como instancia, sino que se los solicita con la función `logging.getLogger` (nombre del logger). Se asigna un nombre concreto al logger, por ejemplo, para mostrar las jerarquías de una manera estructurada. En Python, los directorios de los paquetes se separan con un punto. Por lo tanto, el paquete `log` puede contener los directorios `log.bam` o `log.bar.loco`. Los logger funcionan de manera análoga, de modo que, en este caso, el objeto `log` recibirá los datos de los directorios `log.bam` y `log.bar.loco`.

#### b) Handler
Los handler recopilan la información del logger y la reenvían. El handler es una clase básica que determina cómo actúa la interfaz de las instancias del handler. Para establecer el destino, debes utilizar el tipo de handler correspondiente. **StreamHandler** envía los datos a las secuencias, mientras que **FileHandler** los envía a los archivos. Para un programa, puedes utilizar varios handler que envíen mensajes del mismo logger. Esto te puede ser útil, por ejemplo, si deseas mostrar los datos de depuración en la consola y los mensajes de error importantes en un archivo independiente.

Mediante el método `setLevel()` puedes establecer el nivel mínimo de gravedad que un mensaje de registro requiere para ser reenviado a dicho handler. En lugar de `logger.setLevel` (que determina el nivel de registro), el método recibe el nombre de `[handlername].setLevel`

#### c) Formatter
Los formatter (objetos de formato), a diferencia de los handler, se pueden utilizar **directamente como instancias en el código de la aplicación**. Con estas instancias puedes determinar el formato en el que se emitirá la notificación en el archivo de registro. Si no utilizas ningún formato, solo aparecerá el mensaje especificado del logger. 

#### d) Filter
Los filter permiten crear definiciones aún más precisas para los mensajes de salida. Establece primero los filtros y, después, añádelos al handler o al logger correspondiente mediante el método `addFilter()`. Si el valor de un filtro es false (erróneo) debido a las propiedades del mensaje, no reenviará el mensaje. Utiliza la función `logging.Filter(name = fh)`, donde el atributo fh representa cualquier nombre de logger, para permitir que se envíen únicamente los datos de registro de un logger concreto y bloquear todos los demás logger.

## Logging con Loguru

[Loguru](https://github.com/Delgan/loguru) es una biblioteca que tiene como objetivo brindar un logging agradable en Python.

Además, esta biblioteca está destinada a hacer que el registro de Python sea menos doloroso al agregar un montón de funcionalidades útiles que resuelven las advertencias de los loggers estándar. Usar logs en su aplicación debería ser un automatismo, **Loguru** intenta hacerlo agradable y poderoso.

### Fácil de usar

El concepto principal de "Loguru" es que **hay uno y solo uno** *logger*. Para mayor comodidad, está preconfigurado y se envía a ``stderr`` para empezar (pero eso es completamente configurable).


In [1]:
%%python
from loguru import logger
logger.debug("That's it, beautiful and simple logging!")

2020-09-29 15:17:28.833 | DEBUG    | __main__:<module>:2 - That's it, beautiful and simple logging!


### No Handler, no Formatter, no Filter: todo en uno

¿Cómo agregar un Handler? ¿Cómo configurar el formato de los registros? ¿Cómo filtrar mensajes? ¿Cómo poner el nivel?. Una respuesta: la función `add()`.

In [2]:
%%python
from loguru import logger
import sys
logger.add(sys.stderr, format="{time} {level} {message}", filter="my_module", level="INFO")

Esta función debe usarse para registrar los ``sink``, responsables de administrar los `log messages ` contextualizados con un` record dict`. 

Un ``sink`` puede tomar muchas formas: una función simple, una ruta de cadena, un objeto similar a un archivo, una función de rutina o un controlador integrado (built-in Handler).


### Registro de archivos más fácil con rotation / retention / compression

Si desea enviar mensajes registrados a un archivo, solo tiene que usar una ruta de cadena como ``sink``. También se puede programar automáticamente para mayor comodidad:

In [3]:
%%python
from loguru import logger
logger.add("file_{time}.log")

También es fácilmente configurable si necesita un registrador giratorio (rotating logger), si desea eliminar registros más antiguos o si desea comprimir sus archivos al cierre.

In [4]:
%%python
from loguru import logger
logger.add("file_1.log", rotation="500 MB")    # Automatically rotate too big file
logger.add("file_2.log", rotation="12:00")     # New file is created each day at noon
logger.add("file_3.log", rotation="1 week")    # Once the file is too old, it's rotated
logger.add("file_X.log", retention="10 days")  # Cleanup after some time
logger.add("file_Y.log", compression="zip")    # Save some loved space

### Captura de excepciones dentro `threads` o  `main`

¿Alguna vez ha visto su programa fallar inesperadamente sin ver nada en el archivo de registro? ¿Alguna vez notó que las excepciones que ocurren en los hilos no se registran? Esto se puede resolver usando el decorador/administrador de contexto `catch` que asegura que cualquier error se propague correctamente al `logger`.

In [5]:
%%python

from loguru import logger

@logger.catch
def my_function(x, y, z):
    # An error? It's caught anyway!
    return 1 / (x + y + z)

### Hermosos logging con colores

Loguru agrega automáticamente colores a sus registros si su terminal es compatible. Puede definir su estilo favorito utilizando etiquetas de marcado en el formato `sink`.

In [6]:
%%python
from loguru import logger
import sys
logger.add(sys.stdout, colorize=True, format="<green>{time}</green> <level>{message}</level>")

Las funciones de corrutina utilizadas como `sink` también son compatibles y se deben esperar con `complete()`.

### Excepciones completamente descriptivas

Registrar las excepciones que ocurren en su código es importante para rastrear errores, pero es bastante inútil si no sabe por qué falló. `Loguru` te ayuda a identificar problemas al permitir que se muestre todo el seguimiento de la pila (`stack`), incluidos los valores de las variables (¡gracias `better_exceptions` por esto!).


In [7]:
%%python
from loguru import logger

logger.add("out.log", backtrace=True, diagnose=True)  # Caution, may leak sensitive data in prod

def func(a, b):
    return a / b

def nested(c):
    try:
        func(5, c)
    except ZeroDivisionError:
        logger.exception("What?!")

nested(0)

2020-09-29 15:17:33.143 | ERROR    | __main__:nested:12 - What?!
Traceback (most recent call last):

  File "<stdin>", line 14, in <module>
> File "<stdin>", line 10, in nested
  File "<stdin>", line 6, in func

ZeroDivisionError: division by zero


### Mejor manejo de fecha y hora

el logger estándar está repleto de argumentos como datefmt o `msecs`, `%(asctime)s`y `%(created)s`, fechas y horas ingenuas sin información de zona horaria, formato no intuitivo, etc. Loguru lo corrige:

In [8]:
%%python
from loguru import logger
import sys
logger.add("file.log", format="{time:YYYY-MM-DD at HH:mm:ss} | {level} | {message}")

**Observación**: al final de cada presentación, se eliminan los archivos que generamos de manera temporal.

In [9]:
# eliminar archivos temporales
!rm -r *.log *.zip